# Communes françaises sur carte Folium

- [x] Evaluer la faisabilité de récupérer les data des communes par l'API
- [x] Afficher ces communes sur la carte Folium
- [ ] Donner aux communes une couleur de fond différente
- [x] Optimisation
    - [x] utiliser GeoPandas pour simplifier les polygones avant de faire la carte
    - [x] simplifier affichage si trop de polygones?

## 1. Récupérer les communes françaises

In [ ]:
import json
import logging
from pathlib import Path

import branca
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
from utils import DEPARTEMENTS

logger = logging.getLogger()
logging.basicConfig(level="INFO")

### 1.1. Liste des communes

In [ ]:
def load_communes():
    try:
        response = requests.get("https://geo.api.gouv.fr/communes")
        if response.status_code != 200:
            raise Exception(f"Error when querying geo.api.gouv.fr: {response.status_code}")
        data = json.loads(response.content)
        return data
    except Exception as e:
        raise e

In [ ]:
communes_path = Path("communes.json")

if not communes_path.exists():
    logger.info("Fetch commune data from https://geo.api.gouv.fr/communes")
    communes = load_communes()
    with open(communes_path, "w") as file:
        file.write(json.dumps(communes))
    logger.info(f"Commune data fetched and saved at {communes_path}")
else:
    logger.info("Reading commune data from disk")
    with open(communes_path, "r") as file:
        communes = json.loads(file.read())

In [ ]:
len(communes)

In [ ]:
communes[0]

In [ ]:
# Very long to perform 35000 calls to the API, and not performance-friendly
# Not used

# def get_commune_geom(code: str):
#    response = requests.get(f"https://geo.api.gouv.fr/communes/{code}?fields=contour&format=geojson&geometry=contour")
#    if response.status_code != 200:
#        raise Exception(f"Error when querying geo.api.gouv.fr: {response.status_code}")
#    data = json.loads(response.content)
#    return data

### 1.2. Récupération des géométries par département

In [ ]:
# Get a sorted list of all unique department codes

departements = sorted(list(set(c["codeDepartement"] for c in communes)))

In [ ]:
def load_dept_geojson(codeDept):
    try:
        resp = requests.get(f"https://geo.api.gouv.fr/departements/{codeDept}/communes?format=geojson&geometry=contour")
        if not resp.status_code == 200:
            raise Exception(f"Error when querying geo.api.gouv.fr: {resp.status_code}")
        data = json.loads(resp.content)
        return data
    except Exception as e:
        raise e

In [ ]:
communes_dept_path = Path("communes_par_dept.json")

if not communes_dept_path.exists():
    logger.info("Fetch commune GeoJSON data from https://geo.api.gouv.fr/departements/<codeDept>/communes")
    communes_geojson = [load_dept_geojson(d) for d in departements]

    with open(communes_dept_path, "w") as f:
        f.write(json.dumps(communes_geojson))
    logger.info(f"Commune geographical data fetched and saved at {communes_dept_path}")
else:
    logger.info("Reading commune GeoJSON data from disk")
    with open(communes_dept_path, "r") as f:
        communes_geojson = json.loads(f.read())

### 1.3. Change to GeoDataFrame

In [ ]:
comm_gdf = pd.concat([gpd.GeoDataFrame.from_features(dept) for dept in communes_geojson])

In [ ]:
comm_gdf = comm_gdf.reset_index(drop=True)

In [ ]:
comm_gdf.set_crs("EPSG:3857", inplace=True)

In [ ]:
# Filtrer sur département métropolitains uniquement

dept_domtom = ["971", "972", "973", "974", "975", "976", "977", "978", "984", "986", "987", "988", "989"]

metro_gdf = comm_gdf[~comm_gdf.codeDepartement.isin(dept_domtom)].copy()

In [ ]:
metro_gdf["nomDepartement"] = metro_gdf.codeDepartement.apply(lambda code: DEPARTEMENTS[code])

In [ ]:
metro_gdf

In [ ]:
# Uncomment for map display

metro_gdf.simplify(tolerance=1e-3).plot()

## Display polygons on the map

In [ ]:
metro_departements = metro_gdf["nomDepartement"].unique()

In [ ]:
m = folium.Map(location=[48.85341, 2.3488], zoom_start=7, prefer_canvas=True)

In [ ]:
cmap = branca.colormap.linear.BrBG_03.scale(0, len(metro_departements))
cmap

In [ ]:
for i, dept in enumerate(metro_departements):
    dept_geom = (
        metro_gdf.loc[metro_gdf.nomDepartement == dept, ["nom", "geometry", "nomDepartement"]]
        .reset_index(drop=True)
        .copy()
    )
    # dept_geom.geometry = dept_geom.simplify(tolerance=0.001)

    fg = folium.FeatureGroup(name=dept)

    print(f"{dept} - added {len(dept_geom)} polygons")

    # Version to add all communes individually (with tooltip)
    # for _, row in dept_geom[:3].iterrows():
    #     geoj = folium.GeoJson(
    #         data=row.to_json(),
    #         # style_function=lambda x: {
    #         #    'fillColor': 'orange',
    #         #    "weight": 0,
    #         # },
    #         # smooth_factor=2,
    #     )
    #     folium.features.GeoJsonPopup(fields=["nom"], labels=False).add_to(geoj)
    #     geoj.add_to(fg)

    # Version to add department data at once (but cannot add tooltip with commune name)
    dept_json = dept_geom.simplify(tolerance=0.001).to_json()
    geoj = folium.GeoJson(
        data=dept_json,
        # Styling options for folium are here: https://leafletjs.com/reference.html#path-option
        style_function=lambda x: {"weight": 1, "fillColor": cmap(i)},
        smooth_factor=2,
    )
    geoj.add_to(fg)
    fg.add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
m

In [ ]:
m.save("exported_map_grouped_by_dept.html")

In [ ]:
for _, row in metro_gdf.iterrows():
    sim_geo = gpd.GeoSeries(row["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(
        data=geo_j,
        # style_function=lambda x: {'fillColor': 'orange'},
    )
    folium.Popup(row["nom"]).add_to(geo_j)
    geo_j.add_to(m)

In [ ]:
m

In [ ]:
m.save("exported_map.html")